In [2]:
from bs4 import BeautifulSoup
import requests

In [3]:
url = "https://www.baseball-reference.com/teams/NYY/" # Da Yankees
response = requests.get(url)

In [4]:
response.status_code

200

In [5]:
response.text[:1000]

'\n<!DOCTYPE html>\n<html data-version="klecko-" data-root="/home/br/build" itemscope itemtype="https://schema.org/WebSite" lang="en" class="no-js" >\n<head>\n    <meta charset="utf-8">\n    <meta http-equiv="x-ua-compatible" content="ie=edge">\n    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=2.0" />\n    <link rel="dns-prefetch" href="https://d2p3bygnnzw9w3.cloudfront.net/req/202006261" />\n\n<!-- no:cookie fast load the css.           -->\n<script>function gup(n) {n = n.replace(/[\\[]/, \'\\\\[\').replace(/[\\]]/, \'\\\\]\'); var r = new RegExp(\'[\\\\?&]\'+n+\'=([^&#]*)\'); var re = r.exec(location.search);   return re === null?\'\':decodeURIComponent(re[1].replace(/\\+/g,\' \'));}; document.srdev = gup(\'srdev\')</script>\n<link rel="preconnect" href="https://d2p3bygnnzw9w3.cloudfront.net" crossorigin>\n<link rel="preconnect" href="https://d3k2oh6evki4b7.cloudfront.net" crossorigin>\n <style>html,body{margin:0;padding:0;font:14px/1.25 "Helvet

In [6]:
page = response.text


In [7]:
caldo_feo = BeautifulSoup(page, "lxml")

In [8]:
print(caldo_feo)

<!DOCTYPE html>
<html class="no-js" data-root="/home/br/build" data-version="klecko-" itemscope="" itemtype="https://schema.org/WebSite" lang="en">
<head>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport"/>
<link href="https://d2p3bygnnzw9w3.cloudfront.net/req/202006261" rel="dns-prefetch"/>
<!-- no:cookie fast load the css.           -->
<script>function gup(n) {n = n.replace(/[\[]/, '\\[').replace(/[\]]/, '\\]'); var r = new RegExp('[\\?&]'+n+'=([^&#]*)'); var re = r.exec(location.search);   return re === null?'':decodeURIComponent(re[1].replace(/\+/g,' '));}; document.srdev = gup('srdev')</script>
<link crossorigin="" href="https://d2p3bygnnzw9w3.cloudfront.net" rel="preconnect"/>
<link crossorigin="" href="https://d3k2oh6evki4b7.cloudfront.net" rel="preconnect"/>
<style>html,body{margin:0;padding:0;font:14px/1.25 "Helvetica Neue",helvetica,arial,sans-serif;color:#00

In [9]:
print(caldo_feo.prettify)

<bound method Tag.prettify of <!DOCTYPE html>
<html class="no-js" data-root="/home/br/build" data-version="klecko-" itemscope="" itemtype="https://schema.org/WebSite" lang="en">
<head>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport"/>
<link href="https://d2p3bygnnzw9w3.cloudfront.net/req/202006261" rel="dns-prefetch"/>
<!-- no:cookie fast load the css.           -->
<script>function gup(n) {n = n.replace(/[\[]/, '\\[').replace(/[\]]/, '\\]'); var r = new RegExp('[\\?&]'+n+'=([^&#]*)'); var re = r.exec(location.search);   return re === null?'':decodeURIComponent(re[1].replace(/\+/g,' '));}; document.srdev = gup('srdev')</script>
<link crossorigin="" href="https://d2p3bygnnzw9w3.cloudfront.net" rel="preconnect"/>
<link crossorigin="" href="https://d3k2oh6evki4b7.cloudfront.net" rel="preconnect"/>
<style>html,body{margin:0;padding:0;font:14px/1.25 "Helvetica Neue",helvet

In [10]:

 links =   [print(link,'/n')for link in caldo_feo.find_all('a')]
    

<a href="https://www.sports-reference.com/"><svg height="15px" width="20px"><use xlink:href="#ic-sr-pennant"></use></svg> Sports Reference</a> /n
<a href="https://www.baseball-reference.com/">Baseball</a> /n
<a href="https://www.pro-football-reference.com/">Football</a> /n
<a href="https://www.sports-reference.com/cfb/">(college)</a> /n
<a href="https://www.basketball-reference.com/">Basketball</a> /n
<a href="https://www.sports-reference.com/cbb/">(college)</a> /n
<a href="https://www.hockey-reference.com/">Hockey</a> /n
<a href="https://fbref.com/">Soccer</a> /n
<a href="https://www.sports-reference.com/blog/">Blog</a> /n
<a href="https://stathead.com/?utm_source=web&amp;utm_medium=br&amp;utm_campaign=sr-nav-bar-top-link">Stathead</a> /n
<a href="https://widgets.sports-reference.com/">Widgets</a> /n
<a href="https://www.sports-reference.com/feedback/">Questions or Comments?</a> /n
<a href="https://stathead.com/profile/?utm_source=web&amp;utm_medium=br&amp;utm_campaign=sr-nav-bar-top-

<a href="/players/p/pennohe01.shtml" title="Herb Pennock">H.Pennock</a> /n
<a href="/managers/huggimi01.shtml" title="Miller Huggins">M.Huggins</a> /n
<a href="/teams/NYY/1924.shtml">1924</a> /n
<a href="/teams/NYY/1924.shtml">New York Yankees</a> /n
<a href="/leagues/AL/1924.shtml">AL</a> /n
<a href="/players/r/ruthba01.shtml" title="Babe Ruth">B.Ruth</a> /n
<a href="/managers/huggimi01.shtml" title="Miller Huggins">M.Huggins</a> /n
<a href="/teams/NYY/1923.shtml">1923</a> /n
<a href="/teams/NYY/1923.shtml">New York Yankees</a> /n
<a href="/leagues/AL/1923.shtml">AL</a> /n
<a href="/postseason/1923_WS.shtml">Won WS (4-2)</a> /n
<a href="/players/r/ruthba01.shtml" title="Babe Ruth">B.Ruth</a> /n
<a href="/managers/huggimi01.shtml" title="Miller Huggins">M.Huggins</a> /n
<a href="/teams/NYY/1922.shtml">1922</a> /n
<a href="/teams/NYY/1922.shtml">New York Yankees</a> /n
<a href="/leagues/AL/1922.shtml">AL</a> /n
<a href="/postseason/1922_WS.shtml">Lost WS (4-0-1)</a> /n
<a href="/players

<a href="/register/kbo-stats.shtml">Korean Baseball Stats</a> /n
<a href="/register/league.cgi?year=2019">2019 Minor Leagues</a> /n
<a href="/register/affiliate.cgi?year=2019">2019 Affiliates</a> /n
<a href="/bullpen/">Baseball-Reference Bullpen</a> /n
<a href="/bullpen/">85,000+ pages of baseball information</a> /n
<a href="/bullpen/How_to_contribute">How to Contribute</a> /n
<a href="/about/">About Baseball Reference</a> /n
<a href="/about/bat_glossary.shtml">Batting Glossary</a> /n
<a href="/about/pitch_glossary.shtml">Pitching Glossary</a> /n
<a href="/about/war_explained.shtml">Wins Above Replacement Explainer</a> /n
<a href="/data/">WAR data archive</a> /n
<a href="/about/coverage.shtml">BR Data Coverage</a> /n
<a class="poptip" data-label="Telephone" data-tip="Call us on Telephone" href="tel://888-512-8907"><svg class="icon phone"><use xlink:href="#ic-phone"></use></svg></a> /n
<a class="poptip" data-label="Our Blog" data-tip="The Sports Reference Blog" href="//www.sports-refere

In [11]:
for element in caldo_feo.find_all(data-stat='year_id'):
    print(element.prettify)

SyntaxError: keyword can't be an expression (<ipython-input-11-8076648ac117>, line 1)

In [ ]:
#franchise_years > tbody > tr:nth-child(23)
#.find('tr',{'data-row':'1'})

In [13]:
year_2019 = caldo_feo.find('tbody').find('tr',{'data-row':'1'})

In [15]:
type(year_2019)


NoneType

In [31]:
years = caldo_feo.find('tbody')

In [32]:
years.find_all('tr')[1]

<tr><th class="left" data-stat="year_ID" scope="row"><a href="/teams/NYY/2019.shtml">2019</a></th><td class="left" data-stat="team_name"><a href="/teams/NYY/2019.shtml">New York Yankees</a></td><td class="left" data-stat="lg_ID"><a href="/leagues/AL/2019.shtml">AL East</a></td><td class="right" data-stat="G">162</td><td class="right" data-stat="W">103</td><td class="right" data-stat="L">59</td><td class="right iz" data-stat="ties">0</td><td class="right" data-stat="win_loss_perc">.636</td><td class="right" data-stat="win_loss_perc_pythag">.610</td><td class="right" csk="1" data-stat="finish">1st of 5</td><td class="right" csk="0.0" data-stat="games_back">--</td><td class="right" csk="6" data-stat="playoffs"><a href="/postseason/2019_ALCS.shtml">Lost ALCS (4-2)</a></td><td class="right" data-stat="R">943</td><td class="right" data-stat="RA">739</td><td class="right" data-stat="attendance">3,304,404</td><td class="right" data-stat="age_bat">28.3</td><td class="right" data-stat="age_pit">

In [46]:
franchise_history = caldo_feo.find('table',{'id':'franchise_years'}).find('tbody').find_all('tr')[1]
franchise_history.find('td',{'data-stat':'win_loss_perc'}).text

    

'.636'

In [33]:
data_site = 'https://www.baseball-reference.com'
page_location = '/teams/NYY/2019.shtml'
page = data_site + page_location
page

'https://www.baseball-reference.com/teams/NYY/2019.shtml'